In [47]:
import numpy as np
import pandas as pd
import math
import time

In [48]:
#Cálculo1,LPT,P1,IC,Cálculo2,cra 
#names=["size","bedroom","price"]
rdata = pd.read_csv('sample_treino.csv') #read the data
#rdata = pd.read_csv('home.txt',names=["size","bedroom","price"])
#rdata = (rdata - rdata.mean())/rdata.std()
#
data = rdata.iloc[:,0:len(rdata.columns)-1]
#add extra columns with ones
data = np.c_[np.ones(len(rdata)),data]

In [49]:
#data.iloc[:,2:3].values # y=.values converts it from pandas.core.frame.DataFrame to numpy.ndarray
y = rdata.iloc[:,len(rdata.columns)-1:len(rdata.columns)].values
theta = np.zeros([1,len(rdata.columns)])

$ RSS(w) = \frac{1}{N} \sum_{i=1}^{N} {(X^T \cdot W - Y_i)^2}$

In [50]:
def MSE(X,Y,w):
    return np.sum(np.power(((X @ w.T)-Y),2))/len(X)

In [51]:
print(MSE(data,y,theta))

54.47995385562645


$ w^{t+1} = w^t -2H^t (y-H_w)$

In [52]:
def step_gradient(w,X,Y,learningRate):
    #return w - ((learningRate/len(X)) * np.sum(X * (X @ w.T - Y), axis=0) )
    return w - ((learningRate) * np.sum(X * (X @ w.T - Y), axis=0) )

In [39]:
def gradient_descent_runner(starting_w,X,Y, learning_rate, epsilon):
    w = starting_w
    w_old = [np.inf]
    grad = np.array([np.inf])
    i = 0
    #while ( (np.linalg.norm(grad)>=epsilon) and (np.linalg.norm(grad) <= np.linalg.norm(np.array(w_old))) ):
    while (np.linalg.norm(grad)>=epsilon):
        w_old = w
        w = step_gradient(w, X, Y, learning_rate)
        #print('{:.3f}'.format(np.linalg.norm(grad)),np.linalg.norm(grad)>=epsilon)
        grad = np.array(w)
        #print(np.linalg.norm(grad))
        i+= 1
    print(np.linalg.norm(grad))
    return w,i

gradient_descent_runner(theta,data,y,0.01,0.04)

KeyboardInterrupt: 

In [45]:
def gradient_descent_runner(starting_w,X,Y, learning_rate, epsilon):
    w = starting_w
    w_old = [np.inf]
    grad = np.array([np.inf])
    #print(np.linalg.norm(grad))
    #print(np.linalg.norm(np.array(w_old)))
    i = 0
    while (i < 1000):
    #while (np.linalg.norm(grad) <= np.linalg.norm(np.array(w_old))):
        w_old = w
        w = step_gradient(w, X, Y, learning_rate)
        #print('{:.3f}'.format(np.linalg.norm(grad)),np.linalg.norm(grad)>=epsilon)
        grad = np.array(w)
        if (i % 100 == 0):
            print(np.linalg.norm(grad))
        i+= 1
    #print("sss",np.linalg.norm(grad))
    return w,i

In [46]:
ws,i = gradient_descent_runner(theta,data,y,0.01,0.879)
print(np.linalg.norm(np.array(ws)))
print(ws,i)
print(MSE(data, y,ws))
#[[-1.10868761e-16  8.78503652e-01 -4.69166570e-02]] 

0.4427959807964417
0.8863627022952231
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
0.8863627023548142
[[-1.10467191e-16  8.84765988e-01 -5.31788197e-02]] 1000
0.26137296107808394


sss

In [111]:
#from sklearn import linear_model
#lm = linear_model.LinearRegression()
#model = lm.fit(X,y)
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=0)

In [148]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
#regressor.fit(X_train, y_train)
regressor.fit(data, y)
print(regressor.score(data, y))
regressor.coef_
#[rdata.columns]

0.4238032409512109


array([[0.        , 0.10304143, 0.0464367 , 0.16409834, 0.38117843,
        0.02027816]])

In [149]:
#coeff_df = pd.DataFrame(regressor.coef_, rdata.columns[:-1], columns=['Coefficient'])  
#coeff_df  
#y_pred = regressor.predict(X_test)
y_pred = regressor.predict(data)
#df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
#df

In [150]:
from sklearn import metrics 
print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, y_pred)))

Mean Absolute Error: 0.4898513745150918
Mean Squared Error: 0.41133758922770963
Root Mean Squared Error: 0.6413560549552094


In [143]:
from sklearn import metrics 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))



ValueError: Found input variables with inconsistent numbers of samples: [18, 10]

In [ ]:
#gradient descent
def gradientDescent(X,y,theta,iters,alpha):
    mse = np.zeros(iters)
    for i in range(iters):
        
        msg[i] = MSE_ok(X, y, theta)
    
    return theta,cost

#running the gd and cost function
g,cost = gradientDescent(X,y,theta,iters,alpha)
print(g)

finalCost = computeCost(X,y,g)
print(finalCost)

In [ ]:
def step_gradient_vectorized(w_current,X,Y,learningRate):
    res = Y - np.dot(X,w_current)
    b_gradient = np.sum(res)
    X = X[:,1][:,np.newaxis]
    m_gradient = np.sum(np.multiply(res,X))
    new_w = np.array([(w_current[0] + (2 * learningRate * b_gradient)),
             (w_current[1] + (2 * learningRate * m_gradient))])
    return [new_w,b_gradient,m_gradient]

$MSE(\hat{w})=\frac{1}{N}(y-\hat{\mathbf{w}}^T\mathbf{x})^T(y-\hat{\mathbf{w}}^T\mathbf{x})$

In [ ]:
def compute_mse_vectorized(w,X,Y):
    res = Y - np.dot(X,w)
    totalError = np.dot(res.T,res)
    return totalError / float(len(Y))

In [ ]:
def step_gradient_vectorized(w_current,X,Y,learningRate):
    res = Y - np.dot(X,w_current)
    b_gradient = np.sum(res)
    X = X[:,1][:,np.newaxis]
    m_gradient = np.sum(np.multiply(res,X))
    new_w = np.array([(w_current[0] + (2 * learningRate * b_gradient)),
             (w_current[1] + (2 * learningRate * m_gradient))])
    return [new_w,b_gradient,m_gradient]

In [ ]:
def gradient_descent_runner_vectorized(starting_w, X,Y, learning_rate, epsilon):
    w = starting_w
    grad = np.array([np.inf,np.inf])
    i = 0
    while (np.linalg.norm(grad)>=epsilon):
        w,b_gradient,m_gradient = step_gradient_vectorized(w, X, Y, learning_rate)
        grad = np.array([b_gradient,m_gradient])
        #print(np.linalg.norm(grad))
        if i % 1000 == 0:
            print("MSE na iteração {0} é de {1}".format(i,compute_mse_vectorized(w, X, Y)))
        i+= 1
    return w

In [ ]:
points = np.genfromtxt("income.csv", delimiter=",")
points = np.c_[np.ones(len(points)),points]
X = points[:,[0,1]]
Y = points[:,2][:,np.newaxis]
init_w = np.zeros((2,1))
learning_rate = 0.0001
#num_iterations = 10000
epsilon = 0.5
print("Starting gradient descent at w0 = {0}, w1 = {1}, error = {2}".format(init_w[0], init_w[1], compute_mse_vectorized(init_w, X,Y)))
print("Running...")
tic = time.time()
w = gradient_descent_runner_vectorized(init_w, X,Y, learning_rate, epsilon)
toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, error = {2}".format(w[0], w[1], compute_mse_vectorized(w,X,Y)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")
